In [10]:
import requests
import json
import pandas as pd
import math

base_url = 'https://fantasy.premierleague.com/api/'


def get_transfers(league):
    # Get manager details
    general_info = requests.get(
        base_url + "leagues-h2h-matches/league/" + str(league) + "/").json()

    # For each manager id, get the the details
    # for id_ in manager_ids:
    manager_details_ = [{"manager_id": manager['entry_1_entry'], "player_name": manager['entry_1_player_name'],
                         "name": manager['entry_1_name']} for manager in general_info['results']]
    manager_details = list(
        {manager['manager_id']: manager for manager in manager_details_}.values())

    # For each manager, get their pick history
    manager_picks = []
    for manager in manager_details:
        # For leagues with teams, a virtual manager is include.
        # Check if a manager_id is 'None' and skip that manager
        if (manager['manager_id'] == None):
            continue

        picks = []
        for gw in range(1, 39):
            try:
                gw_picks = requests.get(
                    base_url + "entry/" + str(manager['manager_id']) + "/event/" + str(gw) + "/picks/").json()
            except requests.exceptions.JSONDecodeError as jsonerror:
                print("JSONEconding Error: ", jsonerror, ", ",
                      gw, '=========> ', manager['manager_id'])
            except requests.exceptions.HTTPError as errh:
                print("Http Error:", errh)
            except requests.exceptions.ConnectionError as errc:
                print("Error Connecting:", errc)
            except requests.exceptions.Timeout as errt:
                print("Timeout Error:", errt)
            except requests.exceptions.RequestException as err:
                print("OOps: Something Else", err)

        picks = []
        for gw in range(1, 39):
            gw_picks = requests.get(
                base_url + "entry/" + str(manager['manager_id']) + "/event/" + str(gw) + "/picks/").json()
            picks.append({gw: gw_picks})
        manager_picks.append({manager['manager_id']: picks})

    return {league: manager_picks}

In [ ]:
# Get league ids between 2123000 and 2123881
league_ids = []
for i in range(2125120, 2126441):  # 2123882, 2124764 ,  2124089
    general_info = requests.get(
        base_url + "leagues-h2h-matches/league/" + str(i) + "/").json()
    # Check if the response contains a results list and it is not empty
    if ('results' in general_info and general_info['results']):
        league_ids.append(i)
print('leagues collected')
# league_history = [get_transfers(league) for league in league_ids]
len(league_ids)

In [ ]:
for league in league_ids:
    print('==> Fetching league transfer details for ', league)
    leagues_transfer_details = get_transfers(league)

    print('<== Completed fetching league transfer details for ', league)
    # Append the manager league details to a json file
    data_file_contents = []
    with open("/content/drive/MyDrive/Colab Notebooks/h2h_league_picks_5.json") as data_file:
        data_file_contents = json.load(data_file)

    # # parsed_json = json.loads(file_contents)
    with open("/content/drive/MyDrive/Colab Notebooks/h2h_league_picks_5.json", mode='w',  encoding='utf-8') as f:
        data_file_contents.append(leagues_transfer_details)
        json.dump(data_file_contents, f)

    # Log some details to show progress
    print(league, ':::::::::::',  round(
        (league_ids.index(league)+1)/len(league_ids)*100, 1), "%", " done")

In [11]:
import json
import pandas as pd
data_file_contents = []
matches_data_file = []

# with open("./content/h2h_league/h2h_league_picks_5.json", mode='r') as data_file:
#     data_file_contents = json.load(data_file)

with open("./content/h2h_matches.json", mode='r') as data_file:
    matches_data_file = json.load(data_file)

In [52]:
v = []
with open("./content/h2h_league/unti5/h2h_league_picks_1.json", mode='r') as data_file:
    v = json.load(data_file)

JSONDecodeError: Expecting ':' delimiter: line 1 column 231152623 (char 231152622)

In [42]:
# league_id, manager_id, gw, active_chip, automatic_subs, gw_points, total_points, picks: (), captain, vice_captain, bank, opponent, opponent_points, results,

managers = []


# league_keys = [list(league.keys())[0] for league in data_file_contents]
for league in data_file_contents:
    # for match in matches_data_file:
    #     print(int(list(league.keys())[0]), match)
    # # league_matches = list(list(filter(lambda x: int(list(x.keys())[0]) == int(list(league.keys())[0]), matches_data_file))[0].values())[0]

    for manager in league[list(league.keys())[0]]:
        for gw in manager[list(manager.keys())[0]]:

            active_chip = gw[list(gw.keys())[0]]['active_chip']
            automatic_subs = [(i['element_in'], i['element_out'])
                              for i in gw[list(gw.keys())[0]]['automatic_subs']]
            print(automatic_subs)
            gw_points = gw[list(gw.keys())[0]]['entry_history']['points']
            total_points = gw[list(gw.keys())[0]
                              ]['entry_history']['total_points']

            picks = tuple([player['element']
                           for player in gw[list(gw.keys())[0]]['picks']])
            captain = None
            vice_captain = None

            for player in gw[list(gw.keys())[0]]['picks']:
                if (player['is_captain']):
                    captain = player['element']
                if (player['is_vice_captain']):
                    vice_captain = player['element']

            bank = gw[list(gw.keys())[0]]['entry_history']['bank']
            manager_history = {"league_id": list(league.keys())[0], 'manager_id': list(manager.keys())[0], 'gw': list(gw.keys())[0],
                               'active_chip': active_chip, 'automatic_subs': automatic_subs, 'gw_points': gw_points, 'total_points': total_points, 'picks': picks, 'captain': captain, 'vice_captain': vice_captain, 'bank': bank}
            managers.append(manager_history)


managers

[(302, 4)]
[]
[(119, 302)]
[]
[]
[(616, 401), (5, 302)]
[(302, 405)]
[(616, 405), (302, 195)]
[(616, 405), (302, 195)]
[(616, 405), (302, 195)]
[]
[(119, 302)]
[(119, 302), (108, 143)]
[(524, 291)]
[(119, 13)]
[(427, 108), (365, 355)]
[(427, 108), (217, 355)]
[(524, 352), (427, 108), (217, 355)]
[(427, 302), (217, 108), (365, 143)]
[(427, 108), (217, 143), (365, 355)]
[(427, 108)]
[(427, 108)]
[(427, 108)]
[(427, 302), (143, 108)]
[(427, 217), (143, 108)]
[(427, 302)]
[(427, 217)]
[(308, 217)]
[(108, 427)]
[(108, 217), (308, 700)]
[(220, 19), (108, 700), (242, 355)]
[(220, 302)]
[(352, 524), (220, 365)]
[(365, 13), (108, 355)]
[(365, 302), (108, 242)]
[(365, 427), (108, 302)]
[(220, 302), (108, 13)]
[(439, 302), (468, 412)]
[]
[]
[(263, 148), (415, 258)]
[]
[(407, 131), (258, 372), (220, 415)]
[(407, 103), (220, 372)]
[]
[(197, 131)]
[(197, 131), (412, 437)]
[(197, 203)]
[]
[]
[(341, 203)]
[]
[(263, 524), (60, 13)]
[(409, 524), (664, 355)]
[(143, 355), (220, 415)]
[(260, 584)]
[(260, 5

KeyError: 'active_chip'

In [43]:
managers_pd = pd.DataFrame(managers)

In [44]:
managers_pd.to_csv('./content/managers.csv')

In [73]:
import json_stream

data_path = "content/h2h_league_picks.json"

# data = json_stream.load(data_path)

df = pd.read_json(data_path, lines=True)

ValueError: No ':' found when decoding object value

In [3]:
v

[{1: {'active_chip': None,
   'automatic_subs': [],
   'entry_history': {'event': 1,
    'points': 80,
    'total_points': 80,
    'rank': 710070,
    'rank_sort': 712191,
    'overall_rank': 710070,
    'percentile_rank': None,
    'bank': 5,
    'value': 1000,
    'event_transfers': 0,
    'event_transfers_cost': 0,
    'points_on_bench': 8},
   'picks': [{'element': 263,
     'position': 1,
     'multiplier': 1,
     'is_captain': False,
     'is_vice_captain': False},
    {'element': 195,
     'position': 2,
     'multiplier': 1,
     'is_captain': False,
     'is_vice_captain': False},
    {'element': 131,
     'position': 3,
     'multiplier': 1,
     'is_captain': False,
     'is_vice_captain': False},
    {'element': 5,
     'position': 4,
     'multiplier': 1,
     'is_captain': False,
     'is_vice_captain': False},
    {'element': 143,
     'position': 5,
     'multiplier': 1,
     'is_captain': False,
     'is_vice_captain': False},
    {'element': 308,
     'position': 6,


## Get Match details


In [23]:
# league_id, manager_id, gw, active_chip, automatic_subs, gw_points, total_points, picks: (), captain, vice_captain, bank, opponent, opponent_points, results,

matches_details = []


# league_keys = [list(league.keys())[0] for league in data_file_contents]
for league in matches_data_file:
    print(list(league.keys())[0])
    # for match in matches_data_file:
    #     print(int(list(league.keys())[0]), match)
    # # league_matches = list(list(filter(lambda x: int(list(x.keys())[0]) == int(list(league.keys())[0]), matches_data_file))[0].values())[0]

    for manager in league[list(league.keys())[0]]:
        for gw in manager[list(manager.keys())[0]]:

            team1 = gw['entry_1_entry']
            score1 = gw['entry_1_points']
            team2 = gw['entry_2_entry']
            score2 = gw['entry_2_points']

            matches_details.append({'League': list(league.keys())[0], 'Game Week': list(manager.keys())[
                                   0],  'Team 1': team1, 'Score 1': score1, 'Team 2': team2, 'Score 2': score2})


pd.DataFrame(matches_details)

2123002
2123013
2123014


,League,Game Week,Team 1,Score 1,Team 2,Score 2
0,2123002,4,1722996,82,305196,61
1,2123002,4,1040512,81,2174524,66
2,2123002,5,305196,39,1040512,36
3,2123002,5,2174524,30,1722996,43
4,2123002,6,2174524,83,305196,60
...,...,...,...,...,...,...
380,2123014,34,9380781,122,7673849,99
381,2123014,35,9380781,58,7673849,75
382,2123014,36,9380781,93,7673849,94
383,2123014,37,9380781,121,7673849,125


In [13]:
matches_data_file

[{'2123002': [{'1': []},
   {'2': []},
   {'3': []},
   {'4': [{'id': 50408387,
      'entry_1_entry': 1722996,
      'entry_1_name': 'RoOFa',
      'entry_1_player_name': 'Raouf Hamas',
      'entry_1_points': 82,
      'entry_1_win': 1,
      'entry_1_draw': 0,
      'entry_1_loss': 0,
      'entry_1_total': 3,
      'entry_2_entry': 305196,
      'entry_2_name': 'Aymen',
      'entry_2_player_name': 'aymen isli',
      'entry_2_points': 61,
      'entry_2_win': 0,
      'entry_2_draw': 0,
      'entry_2_loss': 1,
      'entry_2_total': 0,
      'is_knockout': False,
      'league': 2123002,
      'winner': None,
      'seed_value': None,
      'event': 4,
      'tiebreak': None,
      'is_bye': False,
      'knockout_name': ''},
     {'id': 50408388,
      'entry_1_entry': 1040512,
      'entry_1_name': 'FC Wahab',
      'entry_1_player_name': 'Wahab Baziz',
      'entry_1_points': 81,
      'entry_1_win': 1,
      'entry_1_draw': 0,
      'entry_1_loss': 0,
      'entry_1_total': 3